In [1]:
from settings import *

from src.data.preprocessing import preprocess
import pandas as pd


In [ ]:
# OPEN NEEDED `*.csv` files
ais_train = pd.read_csv(AIS_TRAIN, sep='|')
ais_train['time'] = pd.to_datetime(ais_train['time'])

ais_test = pd.read_csv(AIS_TEST, sep=",")
ais_test['time'] = pd.to_datetime(ais_test['time']) 

X_train, X_val, y_train, y_val, test_set, scaler = preprocess(
    ais_train, 
    ais_test,
    seq_type="n_in_m_out",
    seq_len=10,
    seq_len_out=3,
    verbose=True,
    to_torch=True
)

In [9]:
import torch
from torch import nn
from typing import Dict, Union

if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.set_per_process_memory_fraction(0.)
else:
    device = torch.device("cpu")
device = torch.device("cpu")

dim_ffn = 2048 //4
d_model = 512 // 4

# transformer_params = {
#     "d_model": d_model,
#     "nhead": 8,
#     # "num_encoder_layers": 6,
#     "num_decoder_layers": 2,
#     "dim_feedforward": dim_ffn,
#     "dropout": 0.1,
#     # "activation": str | ((Tensor) -> Tensor) = F.relu,
#     "custom_encoder": None,
#     "custom_decoder": None,
#     "layer_norm_eps": 0.00001,
#     "batch_first": False,
#     "norm_first": False,
#     "bias": True,
#     "device": None,
# }

transformer_decoder_params = {
    "d_model": d_model,
    "nhead": 8,
    # "num_encoder_layers": 6,
    # "num_decoder_layers": 2,
    "dim_feedforward": dim_ffn,
    "dropout": 0.1,
    # "activation": str | ((Tensor) -> Tensor) = F.relu,
    "layer_norm_eps": 0.00001,
    "batch_first": True,
    "norm_first": False,
    # "bias": True,
    "device": device,
}

In [10]:
class DecoderModel(nn.Module):
    def __init__(
            self,
            decoder_params: Dict[int,Union[int, float, bool]] = transformer_decoder_params, 
            num_features: int = 7, 
            num_outputs: int = 6, 
            num_layers: int = 1,
            act_out: nn.Module | None = None
        ) -> None:
        super().__init__()
        self.emb_layer = nn.Linear(num_features, d_model)
        dec_layer = nn.TransformerDecoderLayer(**decoder_params)
        self.model = nn.TransformerDecoder(dec_layer, num_layers=num_layers)
        self.ffn = nn.Linear(dim_ffn, num_outputs)
        self.act_out = act_out # nn.Sigmoid()
        
    def forward(self, x):
        emb = self.emb_layer(x)
        out = self.model(emb, emb)
        if self.act_out:
            return self.act_out(self.ffn(out))
        return self.ffn(out)

In [11]:
from src.train.trainer import Trainer

model = DecoderModel()
trainer = Trainer(
    model=model,
    loss=nn.MSELoss(),
    optimizer=torch.optim.Adam(params=model.parameters()),
    device=device
)

In [ ]:
X_train = torch.Tensor(X_train).to(device)
y_train = torch.Tensor(y_train).to(device)

X_val = torch.Tensor(X_val).to(device)
y_val = torch.Tensor(y_val).to(device)

In [ ]:
model.to(device)

In [ ]:
trainer.fit(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    epochs=2,
    eval_on_test=True
)